In [11]:
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

There are 279 patients having there own unique id

In [12]:
base_path = "dataset/IDC_regular_ps50_idx5/"
folder = os.listdir(base_path)
print(len(folder))

279


In [13]:
from glob import glob
imagePatches = glob('dataset/IDC_regular_ps50_idx5/**/*.png', recursive=True)
print(len(imagePatches)) # total no. of images

277524


In [14]:
for i in range(10):
    print(imagePatches[i])

dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1001_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1051_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1101_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1151_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1201_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1251_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1301_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1351_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1501_class0.png
dataset/IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001_y1551_class0.png


In [15]:
imagePatches[277520]

'dataset/IDC_regular_ps50_idx5\\9383\\1\\9383_idx5_x2051_y951_class1.png'

In [16]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

y = []
for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)
        

In [17]:
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y
images_df.head()

,images,labels
0,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
1,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
2,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
3,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0
4,dataset/IDC_regular_ps50_idx5\10253\0\10253_id...,0


In [21]:
images_df.to_csv('dataset/dataset.csv',index = False) 

In [22]:
images_df.shape

(277524, 2)

In [23]:
images_df.groupby("labels")["labels"].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [ ]:
images_df = pd.read_csv()

In [24]:
# spliting the dataset into train test and validation parts
train, test = train_test_split(images_df, stratify=images_df.labels, test_size=0.33)
print(len(train),len(test))

185941 91583


In [37]:
train

,images,labels
0,dataset/IDC_regular_ps50_idx5\16014\0\16014_id...,0
1,dataset/IDC_regular_ps50_idx5\8975\0\8975_idx5...,0
2,dataset/IDC_regular_ps50_idx5\13617\0\13617_id...,0
3,dataset/IDC_regular_ps50_idx5\14157\0\14157_id...,0
4,dataset/IDC_regular_ps50_idx5\12907\1\12907_id...,1
...,...,...
148747,dataset/IDC_regular_ps50_idx5\12895\1\12895_id...,1
148748,dataset/IDC_regular_ps50_idx5\14153\0\14153_id...,0
148749,dataset/IDC_regular_ps50_idx5\10264\0\10264_id...,0
148750,dataset/IDC_regular_ps50_idx5\9381\0\9381_idx5...,0


In [26]:
train, val = train_test_split(train , stratify=train.labels, test_size=0.2)
print(len(train),len(val))

148752 37189


In [28]:
# saving the data in .csv format in different directories
train.to_csv('dataset/train/train.csv',index=False)
test.to_csv('dataset/test/test.csv',index=False)
val.to_csv('dataset/validation/validation.csv',index=False)

In [29]:
# Reading the dataset
train = pd.read_csv('dataset/train/train.csv')
test = pd.read_csv('dataset/test/test.csv')
validation = pd.read_csv('dataset/validation/validation.csv')

In [33]:
#importing the libraries of pytorch
import torch 
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [34]:
class Transformed_dataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [38]:
# Data preprocessing

trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = Transformed_dataset(df_data=train, transform=trans_train)
dataset_valid = Transformed_dataset(df_data=validation,transform=trans_valid)

In [40]:
loader_train = DataLoader(dataset = dataset_train, batch_size=128, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=64, shuffle=False, num_workers=0)

In [41]:
# making the model architecture
import torch.nn as nn
import torch.nn.functional as F